# Algoritmo KNN
## Por: Matheus Mendonca Lopes, Otávio Santos, Raphael Griffoni, Vinícius Leôncio

Base de dados usada: https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv

Compreendendo a base de dados: <br>![Tabela](\src\tabela.jpg "Tabela")<br>
- A primeira coluna contém o ID de cada veículo. Essa informação não será necessária para a classificação.
- **milhas_por_ano**: representa quantas milhas em média o carro andou por ano.
- **ano_do_modelo**: representa o ano de fabricação do veículo.
- **preco**: representa o preço do veículo.
- **vendido**: rótulo de classificação para o algoritmo.
<br><br>
**Os dados de teste serão os últimos 500 da base de dados do link, os quais serão removidos da base de treino.**

# Definição da Classe do Algoritmo KNN
1. ```__init__```: 
    - Argumentos:
        * K: Representa quantos vizinhos serão pegos pelo algoritmo;
        * x_treino: lista de treino sem a classificação;
        * y_treino: lista de treino contendo classificações.
    - Funcionamento:<br>
        Apenas recebe variáveis básicas para funcionamento do algoritmo.

2. ```previsao```:
    - Argumentos:
        * x_teste: lista com dados que serão testados e classificados
    - Funcionamento:<br>
        Faz uma list comprehension executando a função ```encontra_vizinhos``` enviando cada dado contido na lista como argumento.
        Após terminar, retorna a lista contendo a classificação como 0 ou 1 (representando no ou yes). O rótulo será convertido para 'no' ou 'yes' posteriormente.




In [8]:
class algoritmo_knn: 
    def __init__(self, k, x_treino, y_treino):
        self.k = k
        self.x_treino = x_treino
        self.y_treino = y_treino
        
    def previsao(self, x_teste):
        previsoes = [self.encontra_vizinhos(x_teste_ponto) for x_teste_ponto in x_teste]
        return previsoes
    
    def encontra_vizinhos(self, x_teste_ponto):
        distancias = [self.distancia_euclidiana(x_treino_ponto, x_teste_ponto) for x_treino_ponto in self.x_treino]
        indices_k_vizinhos = sorted(range(len(distancias)), key = lambda i: distancias[i])[:self.k]
        classificacao_k_vizinhos = [self.y_treino[i] for i in indices_k_vizinhos]
        classificacao_mais_comum = max(set(classificacao_k_vizinhos), key=classificacao_k_vizinhos.count)
        return classificacao_mais_comum
        
    def distancia_euclidiana(self, x_treino_ponto, x_teste_ponto):
        distancia = sum((x_treino_col - x_teste_col) ** 2 for x_treino_col, x_teste_col in zip(x_treino_ponto, x_teste_ponto)) ** 0.5
        return distancia

In [9]:
def calcula_milhas_total(milhas_por_ano, ano):
    return milhas_por_ano*(2024-ano)

In [10]:
def prever_vendas(dados_classificados, dados_nao_classificados):
    dados_classificados_filtrados = [(linha[1], linha[2], linha[3], linha[4]) for linha in dados_classificados if linha[4] != '']
    x_treino = [(calcula_milhas_total(float(linha[0]), int(linha[1])), float(linha[2])) for linha in dados_classificados_filtrados]
    y_treino = [linha[-1] for linha in dados_classificados_filtrados]
    
    classificacoes = {'yes': 1, 'no': 0}
    y_treino_classificacoes = [classificacoes[classificacao] for classificacao in y_treino]
    
    knn = algoritmo_knn(k = 3, x_treino = x_treino, y_treino = y_treino_classificacoes)
    
    previsoes = knn.previsao(dados_nao_classificados)
    
    mapeamento_classificacoes = {valor: chave for chave, valor in classificacoes.items()}
    previsoes = [mapeamento_classificacoes[classificacao] for classificacao in previsoes]
    
    return previsoes

In [11]:
dados_classificados = []

with open('car-prices.csv', 'r', encoding='utf-8-sig') as ds_carros:
    linhas = ds_carros.readlines()
    for linha in linhas:
        dados_classificados.append(list(linha.strip().split(',')))


In [12]:
dados_nao_classificados = []

with open('car-no-info.csv', 'r', encoding='utf-8-sig') as ds_teste:
    linhas = ds_teste.readlines()
    dados_nao_classificados_str = []
    for linha in linhas:
        dados_nao_classificados_str.append(list(linha.strip().split(','))[:-1])

    dados_nao_classificados = [(calcula_milhas_total(float(linha[1]), int(linha[2])), float(linha[3])) for linha in dados_nao_classificados_str]

In [13]:
venda_prevista = prever_vendas(dados_classificados, dados_nao_classificados)

In [14]:
dados_previsoes = dados_nao_classificados_str

for i in range(len(dados_previsoes)):
    dados_previsoes[i].append(venda_prevista[i])
    

